In [5]:
import pyscf
import pyscf.tools

from orbitalpartitioning import *

In [6]:
molecule = """
Fe  5.48 1.15 -8.03
S   4.05 -0.61 -8.75
S   7.49 0.42 -9.04
Fe  6.04 -1.22 -9.63
S   5.47 1.25 -5.58
S   4.63 3.28 -8.77
S   5.75 -1.50 -12.05
S   6.86 -3.41 -8.86
C   5.51 4.45 -7.51
H   6.49 4.83 -7.92
H   4.87 5.33 -7.25
H   5.72 3.84 -6.59
C   3.60 1.70 -5.54
H   3.01 0.80 -5.82
H   3.28 2.06 -4.52
H   3.42 2.48 -6.31
C   5.21 -4.22 -9.46
H   5.10 -4.01 -10.55
H   5.21 -5.32 -9.26
H   4.37 -3.72 -8.93
C   7.63 -1.85 -12.24
H   7.90 -2.06 -13.31
H   8.20 -0.96 -11.86
H   7.89 -2.72 -11.59
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol).newton()
# mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

symmetry:  C1


******** <class 'pyscf.scf.rohf.ROHF'> Newton solver flags ********
SCF tol = 1e-08
conv_tol_grad = 1e-05
max. SCF cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_start_cycle = 1
ah_max_cycle = 40
ah_grad_trust_region = 2.5
kf_interval = 4
kf_trust_region = 5
canonicalization = True
max_memory 4000 MB (current use 0 MB)
  HOMO = 0.199929094185757  LUMO = 0.205421971066429
Initial guess E= -5039.93568418248  |g|= 6.96962
macro= 0  E= -5052.88935664801  delta_E= -12.9537  |g|= 6.63009  3 KF 15 JK
macro= 1  E= -5062.3529859656  delta_E= -9.46363  |g|= 2.26532  3 KF 15 JK
macro= 2  E= -5064.43953882975  delta_E= -2.08655  |g|= 0.928833  3 KF 15 JK
macro= 3  E= -5066.29877095334  delta_E= -1.85923  |g|= 0.430438  3 KF 16 JK
macro= 4  E= -5067.13540229638  delta_E= -0.836631  |g|= 0.251609  3 KF 17 JK
macro= 5  E

# Define Fragments by AOs

In [18]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
frag4 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3s", "3p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3s", "3p"):
            frag3.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 3:
        if ao[2] in ("3d"):
            frag4.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3, frag4]
print(frags)


[[14, 15, 16, 17, 18], [33, 38, 39, 40], [51, 56, 57, 58], [81, 82, 83, 84, 85]]


# Define Projectors
We can choose to project onto the non-orthogonal AOs, or onto the symmetrically orthogonalized AOs.

In [19]:
# Define projectors
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])


# Project MOs onto all fragments
For each orbital block (Docc, Sing, Virt), project each subspace onto the full list of fragment AOs. This will determine our full active space.

In [20]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))

 Partition  286 orbitals into a total of   18 orbitals
            Index   Sing. Val. Space       
                0   0.97557941            1*
                1   0.97551133            0*
                2   0.97438008            0*
                3   0.97224832            1*
                4   0.97202480            1*
                5   0.96759246            1*
                6   0.95783509            1*
                7   0.95759599            1*
                8   0.95700853            1*
                9   0.95389459            1*
               10   0.94955095            1*
               11   0.94491514            1*
               12   0.87126110            0*
               13   0.85155192            0*
               14   0.83066280            0*
               15   0.82234829            0*
               16   0.81436863            0*
               17   0.78132897            0*
               18   0.62933799            2
               19   0.58781242            2
   

# Split active space into fragments

In [21]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)


Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


# data["Cact"] = Cact
# data["init_fspace"] = init_fspace 
# data["clusters"] = clusters 
# with open('data_CrOCr.pickle', 'wb') as handle:
#     pickle.dump(data, handle)



 Fragment:  [14, 15, 16, 17, 18]
 Partition   18 orbitals into a total of    5 orbitals
            Index   Sing. Val. Space       
                0   0.96755355            1*
                1   0.96494853            1*
                2   0.94980056            1*
                3   0.93877242            1*
                4   0.92955484            1*
                5   0.25601460            0
                6   0.15108206            0
                7   0.12347333            0
                8   0.10242421            0
                9   0.09045992            0

 Fragment:  [33, 38, 39, 40]
 Partition   18 orbitals into a total of    4 orbitals
            Index   Sing. Val. Space       
                0   0.97440544            0*
                1   0.81974101            0*
                2   0.80211213            0*
                3   0.78897838            0*
                4   0.20410610            1
                5   0.18582698            1
                6   0.173

# Make Integrals

In [22]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)

(286, 80)
(286, 18)
(286, 286)


In [23]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;

In [24]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [25]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [26]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

In [16]:
import numpy as np
Ccmf = np.load("Ccmf.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf.molden", Ccmf)


FileNotFoundError: [Errno 2] No such file or directory: 'Ccmf.npy'